# Please Ssroll down to version 2.0. Version 1.0 will be deleted soon.

#### Version 1.0 below.
___

In [2]:
from gaitlink.lr_detection import UllrichLRDetection
from gaitlink.lr_detection import McCamleyLRDetection
from gaitlink.lr_detection import PretrainedModel
from sklearn import neighbors

from gaitlink.data import load_mobilised_matlab_format, get_all_lab_example_data_paths
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier

from gaitlink.data import LabExampleDataset

/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/gaitlink/data/_mobilised_cvs_dmo_dataset.py:10: UserWarning: Global caching is a little tricky to get right and our implementation is not yet battle-tested. Please double check that the results are correct and report any issues you find.
  from tpcp.caching import hybrid_cache
/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ---------------------------------------------
#           Load some example data
# ---------------------------------------------

dataset = LabExampleDataset(reference_system="INDIP")
datapoint = dataset.get_subset(cohort="HA", participant_id="001", test="Test11", trial="Trial1")

sampling_rate_hz = datapoint.sampling_rate_hz
sampling_rate_hz_ref = datapoint.reference_sampling_rate_hz_

from gaitlink.lr_detection._utils import extract_ref_data

data_list, ic_list, label_list = extract_ref_data(datapoint)

## ML Test
____
Basic usage - loading, setting and retrieving parameters.

Option 1 - using pretrained configurations. A configuration includes:
1) pre-processing filtering parameters
2) an sklearn model, with the corresponding params
3) a scaled used for pre-processing the data

In [4]:
pd_params = UllrichLRDetection.PredefinedParameters.pd
print(pd_params)

# TODO: we should resave the models using the latest sklearn version to avoid the warnings.

Loading pd model and scaler from file...
{'lower_band': 0.5, 'upper_band': 2, 'model': SVC(C=1, kernel='linear'), 'scaler': MinMaxScaler()}


/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.23.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 0.23.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
algo_with_pd_params = UllrichLRDetection(**pd_params)
algo_with_pd_params

UllrichLRDetection(lower_band=0.5, model=SVC(C=1, kernel='linear'), scaler=MinMaxScaler(), upper_band=2)

In [7]:
algo_with_pd_params.model

SVC(C=1, kernel='linear')

In [8]:
algo_with_pd_params.scaler

MinMaxScaler()

In [9]:
algo_with_pd_params.get_params()

{'lower_band': 0.5,
 'model__C': 1,
 'model__break_ties': False,
 'model__cache_size': 200,
 'model__class_weight': None,
 'model__coef0': 0.0,
 'model__decision_function_shape': 'ovr',
 'model__degree': 3,
 'model__gamma': 'scale',
 'model__kernel': 'linear',
 'model__max_iter': -1,
 'model__probability': False,
 'model__random_state': None,
 'model__shrinking': True,
 'model__tol': 0.001,
 'model__verbose': False,
 'model': SVC(C=1, kernel='linear'),
 'scaler__clip': False,
 'scaler__copy': True,
 'scaler__feature_range': (0, 1),
 'scaler': MinMaxScaler(),
 'upper_band': 2}

This is an example of how you might overwrite parameters.

In [10]:
algo_with_pd_params.set_params(model__C = 25, model__gamma = 0.002, scaler__clip = True)
algo_with_pd_params.model

SVC(C=25, gamma=0.002, kernel='linear')

The following method also works.

In [12]:
my_paras = {'model__C': 1.5, 'model__gamma': 0.0024, 'model__kernel': 'linear'}
algo_with_pd_params.set_params(**my_paras)
algo_with_pd_params.model

SVC(C=1.5, gamma=0.0024, kernel='linear')

In [13]:
algo_with_pd_params.get_params()

{'lower_band': 0.5,
 'model__C': 1.5,
 'model__break_ties': False,
 'model__cache_size': 200,
 'model__class_weight': None,
 'model__coef0': 0.0,
 'model__decision_function_shape': 'ovr',
 'model__degree': 3,
 'model__gamma': 0.0024,
 'model__kernel': 'linear',
 'model__max_iter': -1,
 'model__probability': False,
 'model__random_state': None,
 'model__shrinking': True,
 'model__tol': 0.001,
 'model__verbose': False,
 'model': SVC(C=1.5, gamma=0.0024, kernel='linear'),
 'scaler__clip': True,
 'scaler__copy': True,
 'scaler__feature_range': (0, 1),
 'scaler': MinMaxScaler(clip=True),
 'upper_band': 2}

Option 2. Using custom models.
___

In [16]:
custom_algo = UllrichLRDetection(model = svm.SVC())
custom_algo.model

SVC()

In [17]:
custom_algo.set_params(model__C=25, model__gamma=0.002)
custom_algo.model

SVC(C=25, gamma=0.002)

In [18]:
custom_algo.scaler # this should be None

In [19]:
my_paras = {'model__C': 30, 'model__gamma': 0.005, 'model__kernel': 'linear', "upper_band": 3}
custom_algo.set_params(**my_paras)
custom_algo.model

SVC(C=30, gamma=0.005, kernel='linear')

## Using the model for L/R Detection
____
### Option 1 - using a pretrained model.
Please note that this will throw a warning, due to sklearn version inconsistencies. Suggest resaving the model with the current sklearn version used in this package.

In [20]:
hc_params = UllrichLRDetection.PredefinedParameters.hc
print(pd_params)

algo_with_hc_params = UllrichLRDetection(**hc_params)
algo_with_hc_params

algo_with_hc_params.detect(data = data_list,
                          ic_list = ic_list,
                          sampling_rate_hz = sampling_rate_hz)

Loading hc model and scaler from file...
{'lower_band': 0.5, 'upper_band': 2, 'model': SVC(C=1.5, gamma=0.0024, kernel='linear'), 'scaler': MinMaxScaler(clip=True)}
Model checking...


/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.23.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 0.23.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


UllrichLRDetection(lower_band=0.5, model=SVC(C=10, kernel='linear'), scaler=MinMaxScaler(), upper_band=2)

In [22]:
hc_params = UllrichLRDetection.PredefinedParameters.hc # this configuration was already loaded and cached.

Loading cached model and scaler...


Predictions are stored post-execution using the `ic_lr` attribute of the algo instance.

In [28]:
algo_with_hc_params.ic_lr
# note, that the predictions here are not the best...

[  predicted_lr_label
 0              Right
 1               Left
 2               Left
 3              Right
 4               Left
 5              Right
 6               Left,
   predicted_lr_label
 0              Right
 1              Right
 2               Left
 3              Right
 4               Left
 5              Right,
    predicted_lr_label
 0               Right
 1                Left
 2               Right
 3                Left
 4               Right
 5               Right
 6               Right
 7                Left
 8                Left
 9                Left
 10               Left
 11               Left
 12              Right
 13               Left
 14              Right
 15               Left
 16              Right
 17               Left,
    predicted_lr_label
 0                Left
 1                Left
 2                Left
 3               Right
 4               Right
 5                Left
 6               Right
 7                Left
 8               Right


### Option 2 - Using a custom model

The following is expected to throw an error. This is because the model was not fit yet.

In [29]:
custom_algo = UllrichLRDetection(model = svm.SVC())

custom_algo.detect(data = data_list,
                          ic_list = ic_list,
                          sampling_rate_hz = sampling_rate_hz)

Model checking...


RuntimeError: Model is not fitted. Call self_optimize before calling detect.

Fit the model using the `self_optimize` method. Importantly, this will also fit the scaler!

In [30]:
custom_algo.self_optimize(data = data_list,
                      ic_list = ic_list,
                      label_list = label_list,
                      sampling_rate_hz = sampling_rate_hz)

UllrichLRDetection(lower_band=0.5, model=SVC(), scaler=MinMaxScaler(), upper_band=2)

Finally, return the assigned labels.

In [31]:
custom_algo.detect(data = data_list, ic_list = ic_list, sampling_rate_hz = sampling_rate_hz)
custom_algo.ic_lr[0]

Model checking...


,predicted_lr_label
0,Right
1,Right
2,Left
3,Right
4,Left
5,Right
6,Left


Check against reference labels.

In [32]:
label_list[0]

,lr_label
0,Left
1,Right
2,Left
3,Right
4,Left
5,Right
6,Left


### Gridsearch Optimization
_____
We can do optimization using `tpcp`: https://tpcp.readthedocs.io/en/latest/auto_examples/parameter_optimization/_03_gridsearch_cv.html

For this, we need:
1) a dataset
2) a pipeline
3) a scoring method for a parameter search. 

We already have a dataset, this was loaded at the beginning of this tutorial. Let's explore it below:

In [4]:
from gaitlink.lr_detection._utils import extract_ref_data
from gaitlink.data import LabExampleDataset
from sklearn import svm
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score

from gaitlink.lr_detection._lr_optimizer import LROptiPipeline, UllrichLROptimizer

dataset = LabExampleDataset(reference_system="INDIP").get_subset(cohort="HA", test="Test11")
dataset

,cohort,participant_id,time_measure,test,trial
0,HA,001,TimeMeasure1,Test11,Trial1
1,HA,002,TimeMeasure1,Test11,Trial1


### Defining the pipeline

Here, we begin with defining the algo as a basic linear SVM (this is the default option anyway).

Then, we fit the algo to the dataset, using `LROptiPipeline.self_optimize(dataset)`. 

This not only extracts all the relevant features, but it also fits the scaler to ensure that this does not change during the optimization. Otherwise, errors will be thrown.

In [5]:
algo = UllrichLRDetection(svm.SVC(kernel='linear'))

pipe = LROptiPipeline(algo)
pipe.self_optimize(dataset) 

LROptiPipeline(algo=UllrichLRDetection(lower_band=0.5, model=SVC(kernel='linear'), scaler=MinMaxScaler(), upper_band=2))

Testing the pipeline, prior to the optimization. Please note that for now, this can be only ran with a single `datapoint`, i.e. `dataset[0]`. Then, predictions can be retrived using `LROptiPipeline.ic_lr_`

In [6]:
pipe.run(dataset[0])
pipe.ic_lr_


Model checking...


[  predicted_lr_label
 0              Right
 1              Right
 2               Left
 3              Right
 4               Left
 5              Right
 6              Right,
   predicted_lr_label
 0              Right
 1               Left
 2              Right
 3               Left
 4              Right
 5              Right,
    predicted_lr_label
 0                Left
 1               Right
 2                Left
 3               Right
 4                Left
 5               Right
 6                Left
 7               Right
 8                Left
 9               Right
 10               Left
 11              Right
 12               Left
 13              Right
 14               Left
 15              Right
 16               Left
 17              Right,
    predicted_lr_label
 0                Left
 1                Left
 2               Right
 3                Left
 4               Right
 5               Right
 6                Left
 7               Right
 8                Left


### Defining the scorer

In [7]:
from sklearn.metrics import accuracy_score, f1_score

def scoring(pipe, datapoint):
    pipe = pipe.safe_run(datapoint)
    _, _, labels = extract_ref_data(datapoint)

    # accuracy_score expects 1D arrays or lists of lables, so you need to flatten the list of predictions
    flat_labels = [item for sublist in labels for item in sublist.values.flatten()]
    flat_predictions = [item for sublist in pipe.run(datapoint).ic_lr_ for item in sublist.values.flatten()]

    return {"accuracy": accuracy_score(flat_labels, flat_predictions)}

In [8]:
datapoint = dataset[1]
scoring(pipe, datapoint) 

Model checking...
Model checking...


{'accuracy': 0.8043478260869565}

### Finally, we can test the grid search...

In [9]:
from sklearn.model_selection import ParameterGrid

para_grid = ParameterGrid({
    "algo__model__C": [0.5, 0.75, 1, 1.25, 1.5, 2, 5, 7.5, 10, 15]
})

optimizer = UllrichLROptimizer(pipe, para_grid)
optimizer.optimize(dataset, scoring_function = scoring)

Split-Para Combos:   0%|          | 0/20 [00:00<?, ?it/s]

Model checking...


Split-Para Combos:   5%|▌         | 1/20 [00:00<00:09,  2.00it/s]

Model checking...


Model checking...


...

Model checking...


Model checking...


Split-Para Combos: 100%|██████████| 20/20 [00:09<00:00,  2.06it/s]

Model checking...


GridSearchCV(cv=2, n_jobs=None, optimize_with_info=True, parameter_grid=<sklearn.model_selection._search.ParameterGrid object at 0x103667510>, pipeline=LROptiPipeline(algo=UllrichLRDetection(lower_band=0.5, model=SVC(kernel='linear'), scaler=MinMaxScaler(), upper_band=2)), pre_dispatch='n_jobs', progress_bar=True, pure_parameters=False, return_optimized='accuracy', return_train_score=False, safe_optimize=True, scoring=<function scoring at 0x2a3134f40>, verbose=0)

In [10]:
optimizer.optimization_results.best_params_

{'algo__model__C': 10}

In [11]:
optimizer.optimization_results.best_score_

0.782263630089717

In [12]:
pd.DataFrame(optimizer.optimization_results.cv_results_)

,mean_optimize_time,std_optimize_time,mean_score_time,std_score_time,split0_test_data_labels,split1_test_data_labels,split0_train_data_labels,split1_train_data_labels,param_algo__model__C,params,split0_test_accuracy,split1_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_single_accuracy,split1_test_single_accuracy
0,0.117419,0.004923,0.354624,0.026352,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",0.5,{'algo__model__C': 0.5},0.777778,0.717391,0.747585,0.030193,6,[0.7777777777777778],[0.717391304347826]
1,0.116900,0.014179,0.358779,0.026430,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",0.75,{'algo__model__C': 0.75},0.777778,0.760870,0.769324,0.008454,4,[0.7777777777777778],[0.7608695652173914]
2,0.121122,0.008223,0.373579,0.027933,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",1,{'algo__model__C': 1},0.777778,0.717391,0.747585,0.030193,6,[0.7777777777777778],[0.717391304347826]
3,0.117678,0.005729,0.370948,0.023590,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",1.25,{'algo__model__C': 1.25},0.746032,0.739130,0.742581,0.003451,8,[0.746031746031746],[0.7391304347826086]
4,0.115409,0.006887,0.361555,0.030738,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",1.5,{'algo__model__C': 1.5},0.746032,0.739130,0.742581,0.003451,8,[0.746031746031746],[0.7391304347826086]
5,0.114608,0.006863,0.360155,0.016739,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",2,{'algo__model__C': 2},0.746032,0.739130,0.742581,0.003451,8,[0.746031746031746],[0.7391304347826086]
6,0.116866,0.006381,0.369058,0.031072,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",5,{'algo__model__C': 5},0.825397,0.717391,0.771394,0.054003,2,[0.8253968253968254],[0.717391304347826]
7,0.115012,0.006270,0.360365,0.024056,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",7.5,{'algo__model__C': 7.5},0.825397,0.717391,0.771394,0.054003,2,[0.8253968253968254],[0.717391304347826]
8,0.115720,0.007289,0.359855,0.025208,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",10,{'algo__model__C': 10},0.825397,0.739130,0.782264,0.043133,1,[0.8253968253968254],[0.7391304347826086]
9,0.120958,0.003022,0.364546,0.033217,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",15,{'algo__model__C': 15},0.825397,0.695652,0.760524,0.064872,5,[0.8253968253968254],[0.6956521739130435]


____
### Version 1.0 - end
___

# Version 2.0
______
New version with updated call signatires for the detect action methods, modeified scaler placement and further optimizer modifications to handle the updated call signature.

In [2]:
from gaitlink.lr_detection import UllrichLRDetection
from gaitlink.lr_detection import McCamleyLRDetection

from sklearn import neighbors

from gaitlink.data import load_mobilised_matlab_format, get_all_lab_example_data_paths
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier

from gaitlink.data import LabExampleDataset

/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/gaitlink/data/_mobilised_cvs_dmo_dataset.py:10: UserWarning: Global caching is a little tricky to get right and our implementation is not yet battle-tested. Please double check that the results are correct and report any issues you find.
  from tpcp.caching import hybrid_cache
/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ---------------------------------------------
#           Load some example data
# ---------------------------------------------

dataset = LabExampleDataset(reference_system="INDIP")
datapoint = dataset.get_subset(cohort="HA", participant_id="001", test="Test11", trial="Trial1")

sampling_rate_hz = datapoint.sampling_rate_hz
sampling_rate_hz_ref = datapoint.reference_sampling_rate_hz_

from gaitlink.lr_detection._utils import extract_ref_data

data_list, ic_list, label_list = extract_ref_data(datapoint)

In [4]:
# ok, in theory, this should work...

hc_params = UllrichLRDetection.PredefinedParameters.hc

algo_with_hc_params = UllrichLRDetection(**hc_params)
algo_with_hc_params

algo_with_hc_params.detect(data = data_list[0],
                          ic_list = ic_list[0],
                          sampling_rate_hz = sampling_rate_hz)

Loading hc model and scaler from file...


/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.23.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/alexstihi/Documents/PhD_Repos/Mobilize_D/gaitlink/.venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 0.23.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


UllrichLRDetection(lower_band=0.5, model=SVC(C=10, kernel='linear'), scaler=MinMaxScaler(), upper_band=2)

In [5]:
algo_with_hc_params.ic_lr

,predicted_lr_label
0,Right
1,Left
2,Left
3,Right
4,Left
5,Right
6,Left


In [6]:
custom_algo = UllrichLRDetection(model = svm.SVC())

custom_algo.detect(data = data_list[0],
                          ic_list = ic_list[0],
                          sampling_rate_hz = sampling_rate_hz)

RuntimeError: Model is not fitted. Call self_optimize before calling detect.

In [7]:
custom_algo.self_optimize(data_list = data_list,
                          ic_list = ic_list,
                          label_list = label_list,
                          sampling_rate_hz = sampling_rate_hz)

UllrichLRDetection(lower_band=0.5, model=SVC(), scaler=MinMaxScaler(), upper_band=2)

In [8]:
custom_algo.detect(data_list[0], ic_list[0], sampling_rate_hz=100)

UllrichLRDetection(lower_band=0.5, model=SVC(), scaler=MinMaxScaler(), upper_band=2)

In [9]:
custom_algo.ic_lr

,predicted_lr_label
0,Right
1,Right
2,Right
3,Right
4,Right
5,Right
6,Right


In [3]:
from gaitlink.lr_detection._utils import extract_ref_data
from gaitlink.data import LabExampleDataset
from sklearn import svm
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from gaitlink.lr_detection import UllrichLRDetection


from gaitlink.lr_detection._lr_optimizer import LROptiPipeline, UllrichLROptimizer

dataset = LabExampleDataset(reference_system="INDIP").get_subset(cohort="HA", test="Test11")
dataset

,cohort,participant_id,time_measure,test,trial
0,HA,001,TimeMeasure1,Test11,Trial1
1,HA,002,TimeMeasure1,Test11,Trial1


In [4]:
# Define the pipeline:
algo = UllrichLRDetection(svm.SVC(kernel='linear'))

pipe = LROptiPipeline(algo)
pipe.self_optimize(dataset) 

LROptiPipeline(algo=UllrichLRDetection(lower_band=0.5, model=SVC(kernel='linear'), scaler=MinMaxScaler(), upper_band=2))

In [5]:
pipe.run(dataset[0])
pipe.ic_lr_


[  predicted_lr_label
 0               Left
 1               Left
 2               Left
 3              Right
 4               Left
 5              Right
 6               Left,
   predicted_lr_label
 0              Right
 1               Left
 2              Right
 3               Left
 4              Right
 5              Right,
    predicted_lr_label
 0                Left
 1               Right
 2                Left
 3               Right
 4                Left
 5               Right
 6                Left
 7               Right
 8                Left
 9               Right
 10               Left
 11              Right
 12               Left
 13              Right
 14               Left
 15              Right
 16               Left
 17              Right,
    predicted_lr_label
 0                Left
 1                Left
 2               Right
 3                Left
 4               Right
 5               Right
 6                Left
 7               Right
 8                Left


In [6]:
# define the scorer

from sklearn.metrics import accuracy_score, f1_score

def scoring(pipe, datapoint):
    pipe = pipe.safe_run(datapoint)
    _, _, labels = extract_ref_data(datapoint)

    # accuracy_score expects 1D arrays or lists of lables, so you need to flatten the list of predictions
    flat_labels = [item for sublist in labels for item in sublist.values.flatten()]
    flat_predictions = [item for sublist in pipe.run(datapoint).ic_lr_ for item in sublist.values.flatten()]

    return {"accuracy": accuracy_score(flat_labels, flat_predictions)}

In [7]:
datapoint = dataset[1]
scoring(pipe, datapoint) 

{'accuracy': 0.6739130434782609}

In [8]:
from sklearn.model_selection import ParameterGrid

para_grid = ParameterGrid({
    "algo__model__C": [0.5, 0.75, 1, 1.25, 1.5, 2, 5, 7.5, 10, 15]
})

optimizer = UllrichLROptimizer(pipe, para_grid)
optimizer.optimize(dataset, scoring_function = scoring)

Split-Para Combos: 100%|██████████| 20/20 [00:09<00:00,  2.15it/s]


GridSearchCV(cv=2, n_jobs=None, optimize_with_info=True, parameter_grid=<sklearn.model_selection._search.ParameterGrid object at 0x2b3ee5a10>, pipeline=LROptiPipeline(algo=UllrichLRDetection(lower_band=0.5, model=SVC(kernel='linear'), scaler=MinMaxScaler(), upper_band=2)), pre_dispatch='n_jobs', progress_bar=True, pure_parameters=False, return_optimized='accuracy', return_train_score=False, safe_optimize=True, scoring=<function scoring at 0x2b459a8e0>, verbose=0)

In [9]:
optimizer.optimization_results.best_score_

0.7864044168391995

In [11]:
import pandas as pd
pd.DataFrame(optimizer.optimization_results.cv_results_)

,mean_optimize_time,std_optimize_time,mean_score_time,std_score_time,split0_test_data_labels,split1_test_data_labels,split0_train_data_labels,split1_train_data_labels,param_algo__model__C,params,split0_test_accuracy,split1_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_single_accuracy,split1_test_single_accuracy
0,0.114310,0.004433,0.355734,0.024957,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",0.5,{'algo__model__C': 0.5},0.873016,0.673913,0.773464,0.099551,7,[0.873015873015873],[0.6739130434782609]
1,0.111618,0.005097,0.344928,0.023886,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",0.75,{'algo__model__C': 0.75},0.888889,0.673913,0.781401,0.107488,2,[0.8888888888888888],[0.6739130434782609]
2,0.111804,0.009701,0.361952,0.041815,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",1,{'algo__model__C': 1},0.888889,0.673913,0.781401,0.107488,2,[0.8888888888888888],[0.6739130434782609]
3,0.110519,0.008494,0.345713,0.022383,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",1.25,{'algo__model__C': 1.25},0.888889,0.673913,0.781401,0.107488,2,[0.8888888888888888],[0.6739130434782609]
4,0.109003,0.007460,0.345731,0.023578,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",1.5,{'algo__model__C': 1.5},0.904762,0.652174,0.778468,0.126294,5,[0.9047619047619048],[0.6521739130434783]
5,0.109842,0.008787,0.348032,0.026019,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",2,{'algo__model__C': 2},0.904762,0.652174,0.778468,0.126294,5,[0.9047619047619048],[0.6521739130434783]
6,0.111642,0.008102,0.363527,0.042948,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",5,{'algo__model__C': 5},0.888889,0.652174,0.770531,0.118357,8,[0.8888888888888888],[0.6521739130434783]
7,0.110329,0.008403,0.344471,0.024251,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",7.5,{'algo__model__C': 7.5},0.888889,0.652174,0.770531,0.118357,8,[0.8888888888888888],[0.6521739130434783]
8,0.110225,0.007469,0.350697,0.026416,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",10,{'algo__model__C': 10},0.888889,0.652174,0.770531,0.118357,8,[0.8888888888888888],[0.6521739130434783]
9,0.108986,0.007713,0.343161,0.024181,"[(HA, 001, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 002, TimeMeasure1, Test11, Trial1)]","[(HA, 001, TimeMeasure1, Test11, Trial1)]",15,{'algo__model__C': 15},0.920635,0.652174,0.786404,0.134231,1,[0.9206349206349206],[0.6521739130434783]
